In [ ]:
# minist.ipynb
from itertools import product

from src.simulate import simulate
from src.helper import display_train_stats, ExperimentLogger, plot_accuracy
from src.config import *


c_lst = [10, ]
b_lst = [10, 50]
e_lst = [1, 5]

dataset = 'MNIST'
img_dir = f'{img_dir_root}/{dataset}'
stats_dir = f'{stats_dir_root}/{dataset}'

# mlp_iid
for c, b, e in list(product(c_lst, b_lst, e_lst)):
    label_parameters = f"C={c/100: .2f}, B={b},  E={e}"
    print(label_parameters)
    stats, kw = simulate(dataset, num_clients=100, num_participants=c, batch_size=b, num_local_epochs=e, num_rounds=100, learning_rate=0.01, target_accuracy=0.97, iid=True, mlp=True)
    name = f"{dataset}_{kw}_{c}_{b}_{e}"
    fig = display_train_stats(stats, title=name, label=label_parameters, target_accuracy=0.97)
    fig.savefig(f'{img_dir}/{kw}_{c}_{b}_{e}.png')

# mlp_noniid
for c, b, e in list(product(c_lst, b_lst, e_lst)):
    label_parameters = f"C={c/100: .2f}, B={b},  E={e}"
    print(label_parameters)
    stats, kw = simulate(dataset, num_clients=100, num_participants=c, batch_size=b, num_local_epochs=e, num_rounds=100, learning_rate=0.01, target_accuracy=0.97, iid=False, mlp=True)
    name = f"{dataset}_{kw}_{c}_{b}_{e}"
    fig = display_train_stats(stats, title=name, label=label_parameters, target_accuracy=0.97)
    fig.savefig(f'{img_dir}/{kw}_{c}_{b}_{e}.png')

# cnn_iid
for c, b, e in list(product(c_lst[:1], b_lst, e_lst[1:])):
    label_parameters = f"C={c/100: .2f}, B={b},  E={e}"
    print(label_parameters)
    stats, kw = simulate(dataset, num_clients=100, num_participants=c, batch_size=b, num_local_epochs=e, num_rounds=50, learning_rate=0.01, target_accuracy=0.99, iid=True, mlp=False)
    name = f"{dataset}_{kw}_{c}_{b}_{e}"
    fig = display_train_stats(stats, title=name, label=label_parameters, target_accuracy=0.97)
    fig.savefig(f'{img_dir}/{kw}_{c}_{b}_{e}.png')

# cnn_noniid
for c, b, e in list(product(c_lst[:1], b_lst, e_lst[1:])):
    label_parameters = f"C={c/100: .2f}, B={b},  E={e}"
    print(label_parameters)
    stats, kw = simulate(dataset, num_clients=100, num_participants=c, batch_size=b, num_local_epochs=e, num_rounds=50, learning_rate=0.01, target_accuracy=0.99, iid=False, mlp=False)
    name = f"{dataset}_{kw}_{c}_{b}_{e}"
    fig = display_train_stats(stats, title=name, label=label_parameters, target_accuracy=0.97)
    fig.savefig(f'{img_dir}/{kw}_{c}_{b}_{e}.png')


C= 0.10, B=10,  E=1
Dataset MNIST Loaded!
Round 1 | ids: [6, 22, 26, 37, 56, 61, 70, 91, 95, 98] | average accuracy: 0.8582 | train loss: 0.9598 | test loss: 0.5453
Round 2 | ids: [5, 12, 16, 32, 35, 36, 56, 70, 74, 93] | average accuracy: 0.8985 | train loss: 0.7722 | test loss: 0.3589
Round 3 | ids: [13, 17, 26, 38, 50, 55, 62, 74, 76, 95] | average accuracy: 0.9103 | train loss: 0.4434 | test loss: 0.2986
Round 4 | ids: [3, 4, 14, 32, 42, 44, 73, 84, 87, 91] | average accuracy: 0.9206 | train loss: 0.2355 | test loss: 0.2623
Round 5 | ids: [4, 20, 32, 39, 43, 48, 59, 61, 71, 82] | average accuracy: 0.9270 | train loss: 0.4731 | test loss: 0.2425
Round 6 | ids: [3, 6, 8, 10, 14, 18, 22, 55, 57, 59] | average accuracy: 0.9329 | train loss: 0.4168 | test loss: 0.2199
Round 7 | ids: [17, 30, 34, 46, 52, 65, 67, 69, 89, 93] | average accuracy: 0.9379 | train loss: 0.0529 | test loss: 0.2053
Round 8 | ids: [4, 20, 26, 37, 39, 59, 63, 67, 88, 92] | average accuracy: 0.9426 | train loss: 0.

In [ ]:
plot_accuracy(dataset, 'mlp_iid', num_epoch=100)


In [ ]:
plot_accuracy(dataset, 'mlp_noniid', num_epoch=100)

In [ ]:
plot_accuracy(dataset, 'cnn_iid', num_epoch=50)

In [ ]:
plot_accuracy(dataset, 'cnn_noniid', num_epoch=50)